In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re
from importlib import reload
#syntax: fugi = reload(fugi)

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
##update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root");en="550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

# #546eV - 8.2eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root");en="546eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_10800-2350ns_multiCH11.root")

# #544eV - 6.1eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root");en="544eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH11.root")

# # 542eV - 4eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root");en="542eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_10800-2350ns_COMBI_multiCH11.root")

# #541eV 3eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root");en="541eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_10800-2350ns_multiCH11.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[0]+dirs[0]+"/MFPAD_ALLdir_multinew_usr"

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
# d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
# check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# d1=type(check)
# check.all_members; #all members of the single histogram

histRCR = fileRCR[loc]
histRCL = fileRCL[loc]
if en == "550eV":
    histSCR = fileSCR[loc]
    histSCL = fileSCL[loc]
else:
    fileSCR = None
    fileSCL = None

In [ ]:
#backup for the CH9
##550eV - 11.5eV CH9
#update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CR_Knov.root");en="550eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CL_Knov.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CL_Knov.root")

#546eV - 8.2eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_9600-3700ns_multiCH9.root");en="546eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_9600-3700ns_multiCH9.root")

#544eV - 6.1eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH9.root");en="544eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH9.root")

#542eV - 4eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_9600-3700ns_COMBI_multiCH9.root");en="542eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_9600-3700ns_COMBI_multiCH9.root")

#541eV 3eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_9600-3700ns_multiCH9.root");en="541eV";CH="CH9";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_9600-3700ns_multiCH9.root")

In [ ]:
from sklearn.preprocessing import normalize

nhistRCL = histRCL.to_numpy()
r_nhistRCL = np.array(nhistRCL[0], dtype=float) # z values normalise to 1
r_norm_nhistRCL = fugi.normalise_with_err(nhistRCL[0],err=0) # z values normalise to 1
phi_nhistRCL = nhistRCL[1] # phi
costheta_nhistRCL = nhistRCL[2]# cos(theta)

nhistRCR = histRCR.to_numpy()
r_nhistRCR = np.array(nhistRCR[0], dtype=float) # z values normalise to 1
r_norm_nhistRCR = fugi.normalise_with_err(nhistRCR[0],err=0) # z values normalise to 1

totR = np.add(r_nhistRCL,r_nhistRCR)
totR_norm = np.add(r_norm_nhistRCL,r_norm_nhistRCR)
#sum the helicities and flip along cos(theta)
totRf_norm = np.flip(totR_norm,axis=1)

if fileSCL and fileSCR is not None:

    nhistSCR = histSCR.to_numpy()
    r_nhistSCR = np.array(nhistSCR[0], dtype=float) # z values normalise to 1
    r_norm_nhistSCR = fugi.normalise_with_err(nhistSCR[0],err=0) # z values normalise to 1

    r_nhistSCR_flip=np.flip(r_nhistSCR, axis=1)
    r_norm_nhistSCR_flip=np.flip(r_norm_nhistSCR, axis=1)

    nhistSCL = histSCL.to_numpy()
    r_nhistSCL = np.array(nhistSCL[0], dtype=float) # z values normalise to 1
    r_norm_nhistSCL = fugi.normalise_with_err(nhistSCL[0],err=0) # z values normalise to 1
    
    r_nhistSCL_flip=np.flip(r_nhistSCL, axis=1)
    r_norm_nhistSCL_flip=np.flip(r_norm_nhistSCL, axis=1)

    totS = np.add(r_nhistSCL,r_nhistSCR)
    totS_norm = np.add(r_norm_nhistSCL,r_norm_nhistSCR)
    totSf = np.add(r_nhistSCL_flip,r_nhistSCR_flip)
    totSf_norm = np.add(r_norm_nhistSCL_flip,r_norm_nhistSCR_flip)

    #sum all MFPAD: NO NORMALIZATION
    totMFPAD=np.add(totR,totSf)

    #contrast map R-S enantiomer: NORMALISED
    mPECD_norm = np.divide(np.subtract(totR_norm,totS_norm),np.add(totR_norm,totS_norm))
    mPECD_smooth = fugi.smoothgauss(mPECD_norm,0.3,0.6)

else:
    #NOTE no contrast map without the second enantiomer. Maybe one can flip one to simulated it?
    #sum all MFPAD
    totMFPAD=totR

    mPECD_norm = np.divide(np.subtract(totR_norm,totRf_norm),np.add(totR_norm,totRf_norm))
    mPECD_smooth = fugi.smoothgauss(mPECD_norm,0.3,0.6)


cosM = (costheta_nhistRCL[1:] + costheta_nhistRCL[:-1])/2
phiM = (phi_nhistRCL[1:] + phi_nhistRCL[:-1])/2

xgo_phi,ygo_phi = fugi.create_gocoords(a=0,reduced=True)
xgo_cos,ygo_cos = fugi.create_gocoords(a=1,reduced=True)

print(r_norm_nhistRCR.shape)
print(totMFPAD.shape)
print(nhistRCR[0].shape)
print(phi_nhistRCL.shape,costheta_nhistRCL.shape)
print(cosM.shape, phiM.shape)

In [ ]:
if fileSCL and fileSCR is not None:

    fugi = reload(fugi)
    fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
    # cs1=ax1.contourf(phiM,cosM,r_norm_nhistSCL.T, cmap=cmap_temp, levels=15)
    # cs1=ax1.pcolormesh(phiM,cosM,r_norm_nhistSCL.T,shading='gouraud', cmap=cmap_temp)
    cs1,ax1=fugi.plot_interpolation(phiM,cosM,r_norm_nhistSCL.T, ax=ax1, cmap=cmap_temp, xstep=1, ystep=0.005)

    aspect=(phiM.max()/cosM.max())    
    ax1.set_title("MFPAD SCL "+CH+"_"+en)
    ax1.set_xlabel('phi [DEG]')
    ax1.set_ylabel('cos(theta) [adm]')
    ax1.set_aspect(aspect)
    cbar1=fig.colorbar(cs1,extend='both', shrink=0.9, ax=ax1)
    cbar1.set_ticks([r_norm_nhistSCL.min(),r_norm_nhistSCL.max()])
    cbar1.set_ticklabels(["min","max"])
    
    
    cs2=ax2.contourf(phiM,cosM,r_norm_nhistSCL_flip.T,cmap=cmap_temp,levels=15)
    # cs2=ax2.pcolormesh(phiM,cosM,r_norm_nhistSCL_flip.T,shading='gouraud', cmap=cmap_temp)

    ax2.set_title("MFPAD SCL_flip "+CH+"_"+en)
    ax2.set_xlabel('phi [DEG]')
    ax2.set_ylabel('cos(theta) [adm]')
    ax2.set_aspect(aspect)
    cbar2=fig.colorbar(cs2, shrink=0.9, ax=ax2)
    cbar2.set_ticks([r_norm_nhistSCL_flip.min(),r_norm_nhistSCL_flip.max()])
    cbar2.set_ticklabels(["min","max"])
    cbar2.ax.set_ylabel('normalised counts')
    
    # plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
    aspect=(phiM.max()/cosM.max())
    # cs1=ax1.contourf(phiM,cosM,r_norm_nhistSCR.T, cmap=cmap_temp, levels=15)
    # cs1=ax1.pcolormesh(phiM,cosM,r_norm_nhistSCR.T,shading='gouraud', cmap=cmap_temp)
    cs1,ax1=fugi.plot_interpolation(phiM,cosM,r_norm_nhistSCR.T, ax=ax1, cmap=cmap_temp, xstep=1, ystep=0.01)


    ax1.set_title("MFPAD SCR n0 "+CH+"_"+en)
    ax1.set_xlabel('phi [DEG]')
    ax1.set_ylabel('cos(theta) [adm]')
    ax1.set_aspect(aspect)
    cbar1=fig.colorbar(cs1, shrink=0.9, ax=ax1)
    cbar1.set_ticks([r_norm_nhistSCR.min(),r_norm_nhistSCR.max()])
    cbar1.set_ticklabels(["min","max"])

    # cs2=ax2.contourf(phiM,cosM,r_norm_nhistSCL.T,cmap=cmap_temp,levels=15)
    # cs2=ax2.pcolormesh(phiM,cosM,r_norm_nhistSCL.T,shading='gouraud', cmap=cmap_temp)
    cs2,ax2=fugi.plot_interpolation(phiM,cosM,r_norm_nhistSCL.T, ax=ax2, cmap=cmap_temp, xstep=1, ystep=0.01)


    ax2.set_title("MFPAD SCL n0 "+CH+"_"+en)
    ax2.set_xlabel('phi [DEG]')
    ax2.set_ylabel('cos(theta) [adm]')
    ax2.set_aspect(aspect)
    cbar2=fig.colorbar(cs2, shrink=0.9, ax=ax2)
    cbar2.set_ticks([r_norm_nhistSCL.min(),r_norm_nhistSCL.max()])
    cbar2.set_ticklabels(["min","max"])
    cbar2.ax.set_ylabel('normalised counts')

    # fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/SCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", dpi=300, transparent=False)
    # plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
aspect=(phiM.max()/cosM.max())

# cs1=ax1.contourf(phiM,cosM,r_norm_nhistRCR.T, cmap=cmap_temp, levels=15)
# cs1=ax1.pcolormesh(phiM,cosM,r_norm_nhistRCR.T,shading='gouraud', cmap=cmap_temp)
cs1,ax1=fugi.plot_interpolation(phiM,cosM,r_norm_nhistRCR.T, ax=ax1, cmap=cmap_temp, xstep=1, ystep=0.01)


ax1.set_title("MFPAD RCR n0 "+CH+"_"+en)
ax1.set_xlabel('phi [DEG]')
ax1.set_ylabel('cos(theta) [adm]')
ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1, shrink=0.9, ax=ax1)
cbar1.set_ticks([r_norm_nhistRCR.min(),r_norm_nhistRCR.max()])
cbar1.set_ticklabels(["min","max"])

# cs2=ax2.contourf(phiM,cosM,r_norm_nhistRCL.T,cmap=cmap_temp,levels=15)
# cs2=ax2.pcolormesh(phiM,cosM,r_norm_nhistRCL.T,shading='gouraud', cmap=cmap_temp)
cs2,ax2=fugi.plot_interpolation(phiM,cosM,r_norm_nhistRCL.T, ax=ax2, cmap=cmap_temp, xstep=1, ystep=0.01)


ax2.set_title("MFPAD RCL n0 "+CH+"_"+en)
ax2.set_xlabel('phi [DEG]')
ax2.set_ylabel('cos(theta) [adm]')
ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2, shrink=0.9, ax=ax2)
cbar2.set_ticks([r_norm_nhistRCL.min(),r_norm_nhistRCL.max()])
cbar2.set_ticklabels(["min","max"])
cbar2.ax.set_ylabel('normalised counts')

# fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/RCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", dpi=300, transparent=False)
plt.show()

In [ ]:
totMFPAD.shape

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,totMFPAD.T, cmap=cmap_temp, levels=20)
# cs=ax.pcolormesh(phiM,cosM,totMFPAD.T,shading='gouraud', cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,totMFPAD.T, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01)

ax.set_title("MFPAD tot "+CH+"_"+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

# fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/total_MFPAD_"+CH+"_"+en+"_exp.png", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,mPECD_norm.T*100, cmap=cmap_temp)#, levels=24)
# cs=ax.pcolormesh(phiM,cosM,mPECD_norm.T*100,shading='gouraud', vmin=-8.5, vmax=8.5, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_norm.T*100, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01)

ax.set_title("contrast map "+CH+"_"+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([mPECD_norm.min(),mPECD_norm.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel("% of assymetry")
ax.set_aspect(aspect)

# fig.savefig("../PYTHON_graphs/OUTPUTS/TFMOX/contrastmap_R-S_"+CH+"_"+en+"_exp.png", dpi=300, transparent=False)